In [19]:
import gurobipy as gp 
from gurobipy import GRB
import time

In [11]:
import numpy as np
import math
def randInstanceGen(n,m,J,U):
    # Reset seed for safety
    np.random.seed(0)
    # Set seed
    np.random.seed(61836)
    
    c = np.random.randint(1,U+1,(J,n))
    a = np.random.randint(1,U+1,(n,m))
    
    b = np.max(a, axis=0)
    
    for i in range(0, m):
        if math.ceil(np.sum(a[:,i])/2) > b[i]:
            b[i] = math.ceil(np.sum(a[:,i])/2)
    
    return c*-1, a, b

In [48]:
c, a, b = randInstanceGen(70,4,2,40)
print(c)
print("**")
print(a)
print("**")
print(b)

[[-34  -1 -37 -17  -1 -40  -3 -32 -31  -7 -29 -22 -12 -40 -20  -2 -23  -7
   -7 -31 -10 -23  -6 -35 -24 -20 -16 -24 -15 -38 -18  -8  -9  -5 -34 -27
  -35 -24 -21 -23 -33 -22 -27 -10 -23 -16 -31 -18  -4 -40 -34 -14 -29 -10
  -11 -37 -18  -5 -16 -18  -5  -6 -23  -5  -8 -26 -14 -30 -11 -27]
 [-13 -18  -8 -17 -30 -14 -18 -19 -37 -20 -15 -10 -15  -6  -7 -38 -28  -3
   -4 -28  -4 -28 -22 -37  -1 -21  -4  -9 -14 -13 -23  -8 -24 -26 -40  -1
  -10 -15 -30 -22 -36 -25 -32 -32 -12 -18 -32 -16 -27 -40 -35 -11 -16 -25
  -27 -23 -36 -12 -23 -20 -34  -5 -13 -15  -5 -29 -29 -10 -17  -8]]
**
[[10  6 21 34]
 [15  7 24 29]
 [ 1 21  8 24]
 [23 21 24 19]
 [20 27 30 14]
 [ 3 10 16  2]
 [10 34 30 38]
 [35 36  6 17]
 [30 23 39 16]
 [36  4 24 32]
 [16 30 30 35]
 [32 32 36  4]
 [ 7  6 37 39]
 [27 35 33 37]
 [ 4  3 34  1]
 [19 32  2 27]
 [15 29 15 17]
 [18 12  3 16]
 [ 8  7 27 18]
 [16 15 31 22]
 [23 32  7 36]
 [39 30 13  6]
 [23 25  2  9]
 [12 18 28 37]
 [38  9 40 37]
 [25  2 38 13]
 [30 30 31 26]
 [ 3 23 11 11

In [83]:
def rectangular(c,a,b):
    numRect = 1
    found = []
    
    m = gp.Model("Rectangular Method")
    
    n = a.shape[0]
    cons = b.shape[0]
    

    Items=list(np.arange(n))
    Dimensions= list(np.arange(cons))

    # Decision Vars
    x = m.addVars(n,vtype=GRB.BINARY, name="ToTakeOrNotToTake")

    #Mute Output Text
    m.Params.OutputFlag = 0


    #Knapsack Constraints
    for j in Dimensions:
        m.addConstr(gp.quicksum(x[i]*a[i,j] for i in Items) <= b[j], name="j"+str(j))

    #NW 
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    NW = (lexConstraint,hold)
    print(NW)
    
    #SE 
    old = m.getConstrByName('temp')
    m.remove(old)
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    SE = (hold, lexConstraint)
    print(SE)
    
    old = m.getConstrByName('temp')
    m.remove(old)
    
    rectangles = [[NW,SE]]
    
    found.append(list(NW))
    found.append(list(SE))
    
    #points in () , tuples
    #rectangles 2D lists of points 
    #5
    while len(rectangles) != 0:
        #6&7
        R = rectangles.pop(0)
     
        #8
        R2 = [(R[0][0] , (R[0][1]+R[1][1])/2.0), R[1] ] 
        
        
        #lexmin(z1,z2) MAKE SURE IN R2 (going left)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R2[0][1], name = 'top bound')
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.optimize()
        
        z1 = m.objVal
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('top bound')
        m.remove(old)
        #9
        tempNW = (z1,hold) 
        #10/11/12
        if tempNW != R[1]:
            found.append(list(tempNW))
            rectangles.append([tempNW, R[1]])
        #13  
        R3 = [R[0], (tempNW[0] - 1, (R[0][1] + R[1][1])/2.0)]
        
               
               
        #lexmin(z2,z1) = z_opt    
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R3[1][0], name = 'right bound')
        #m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) >= R2[0][1] , name = 'bottom bound')
        
        m.optimize()
        
        z1=m.objVal
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('right bound')
        m.remove(old)
        #old = m.getConstrByName('bottom bound')
        #m.remove(old)
        
        #14
        tempSE = (hold,z1)
        if tempSE != R[0]:
            found.append(list(tempSE))
            rectangles.append([R[0],tempSE])
        numRect += 2
    return found, numRect

In [71]:
timestart = time.perf_counter()
found, numrect = rectangular(c,a,b)
time1 = time.perf_counter() - timestart
print(found, numrect, len(found), len(set(map(tuple,found))))

[[-1032.0, -788.0], [-763.0, -1019.0], [-994.0, -907.0], [-1000.0, -889.0], [-926.0, -969.0], [-927.0, -961.0], [-1024.0, -848.0], [-1029.0, -822.0], [-882.0, -996.0], [-883.0, -993.0], [-966.0, -941.0], [-967.0, -933.0], [-1010.0, -879.0], [-1012.0, -858.0], [-834.0, -1010.0], [-839.0, -1007.0], [-901.0, -981.0], [-907.0, -979.0], [-942.0, -953.0], [-955.0, -949.0], [-976.0, -920.0], [-977.0, -919.0], [-1001.0, -884.0], [-1002.0, -883.0], [-1021.0, -856.0], [-780.0, -1015.0], [-796.0, -1014.0], [-867.0, -1003.0], [-869.0, -998.0], [-889.0, -988.0], [-890.0, -986.0], [-921.0, -978.0], [-922.0, -973.0], [-936.0, -958.0], [-939.0, -956.0], [-956.0, -942.0], [-968.0, -924.0], [-980.0, -913.0], [-983.0, -910.0], [-1004.0, -882.0], [-1005.0, -880.0], [-1013.0, -857.0], [-770.0, -1017.0], [-774.0, -1016.0], [-815.0, -1012.0], [-825.0, -1011.0], [-864.0, -1004.0], [-884.0, -990.0], [-899.0, -984.0], [-900.0, -982.0], [-932.0, -960.0], [-974.0, -923.0], [-978.0, -914.0], [-988.0, -908.0], [-79

In [80]:
def rectangularThirds(c,a,b):
    numRect = 1
    found = []
    
    m = gp.Model("Rectangular Method")
    
    n = a.shape[0]
    cons = b.shape[0]
    

    Items=list(np.arange(n))
    Dimensions= list(np.arange(cons))

    # Decision Vars
    x = m.addVars(n,vtype=GRB.BINARY, name="ToTakeOrNotToTake")

    #Mute Output Text
    m.Params.OutputFlag = 0


    #Knapsack Constraints
    for j in Dimensions:
        m.addConstr(gp.quicksum(x[i]*a[i,j] for i in Items) <= b[j], name="j"+str(j))

    #NW 
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    NW = (lexConstraint,hold)
    
    #SE 
    old = m.getConstrByName('temp')
    m.remove(old)
    m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
    m.optimize()
    lexConstraint = m.objVal
    m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
    m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= lexConstraint, name = 'temp')
    m.optimize()
    hold = m.objVal
    SE = (hold, lexConstraint)
    
    old = m.getConstrByName('temp')
    m.remove(old)
    
    rectangles = [[NW,SE]]
    
    found.append(list(NW))
    found.append(list(SE))
    
    #points in () , tuples
    #rectangles 2D lists of points 
    #5
    while len(rectangles) != 0:
        #6&7
        R = rectangles.pop(0)
     
        #8
        delta = (R[0][1] - R[1][1])/3
        R2 = [(R[0][0] , (R[0][1]-2*delta))    , R[1]]
        
        
        #lexmin(z1,z2) MAKE SURE IN R2 (going left)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R2[0][1], name = 'top bound')
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.optimize()
        
        z1 = m.objVal
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('top bound')
        m.remove(old)
        #9
        tempNW = (z1,hold) 
        #10/11/12
        if tempNW != R[1]:
            found.append(list(tempNW))
            rectangles.append([tempNW, R[1]])
        
        #EXCLUDE BOTTOM, INCLUDE TOP
        
        
        R3 =[    (R[0][0] , R[0][1]-delta)    , (tempNW[0]-1,R2[0][1] + 1)]
        
        #NW 
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) >= R2[0][1]+.01, name = 'bottom bound')
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= R3[0][1], name = 'top bound')
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R3[1][0], name = 'right bound')
        m.optimize()
        if m.status == 2:
            lexConstraint = m.objVal
            m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
            m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= lexConstraint, name = 'temp')
            m.optimize()
            hold = m.objVal
            NW = (lexConstraint,hold)
    
        #SE 
            old = m.getConstrByName('temp')
            m.remove(old)
            m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
            m.optimize()
            lexConstraint = m.objVal
            m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
            m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= lexConstraint, name = 'temp')
            m.optimize()
            hold = m.objVal
            SE = (hold, lexConstraint)
            old = m.getConstrByName('temp')
            m.remove(old)
            old = m.getConstrByName('bottom bound')
            m.remove(old)
            old = m.getConstrByName('top bound')
            m.remove(old)
            old = m.getConstrByName('right bound')
            m.remove(old)
            
            found.append(list(SE))
            if NW != SE:
                found.append(list(NW))
                rectangles.append([NW,SE])
                
        else:
            NW = tempNW
            old = m.getConstrByName('bottom bound')
            m.remove(old)
            old = m.getConstrByName('top bound')
            m.remove(old)
            old = m.getConstrByName('right bound')
            m.remove(old)
        
        
        #13
        #KEEP tempNW[0] IN MIND TO BE CHANGED BY R3 (MIDDLE)
        R4 = [R[0],      (NW[0] - 1, R[0][1]-delta)]
        
               
        #lexmin(z2,z1) = z_opt    
        m.setObjective(gp.quicksum(x[i]*c[1,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[0,i] for i in Items) <= R4[1][0], name = 'right bound')
        
        m.optimize()
        z1=m.objVal
        
        m.setObjective(gp.quicksum(x[i]*c[0,i] for i in Items), GRB.MINIMIZE)
        m.addConstr(gp.quicksum(x[i]*c[1,i] for i in Items) <= z1, name = 'temp')
        m.optimize()
        hold = m.objVal
        old = m.getConstrByName('temp')
        m.remove(old)
        old = m.getConstrByName('right bound')
        m.remove(old)
        #old = m.getConstrByName('bottom bound')
        #m.remove(old)
        
        #14
        tempSE = (hold,z1)
        if tempSE != R[0]:
            found.append(list(tempSE))
            rectangles.append([R[0],tempSE])
        numRect += 3
    return found, numRect

In [81]:
timestart = time.perf_counter()
found, numrect = rectangularThirds(c,a,b)
time2 = time.perf_counter() - timestart
print(found, numrect, len(found), len(set(map(tuple,found))))

[[-1032.0, -788.0], [-763.0, -1019.0], [-956.0, -942.0], [-966.0, -941.0], [-1010.0, -879.0], [-1012.0, -858.0], [-882.0, -996.0], [-883.0, -993.0], [-926.0, -969.0], [-927.0, -961.0], [-974.0, -923.0], [-976.0, -920.0], [-994.0, -907.0], [-1000.0, -889.0], [-1024.0, -848.0], [-1029.0, -822.0], [-815.0, -1012.0], [-825.0, -1011.0], [-864.0, -1004.0], [-867.0, -1003.0], [-894.0, -985.0], [-899.0, -984.0], [-921.0, -978.0], [-922.0, -973.0], [-939.0, -956.0], [-942.0, -953.0], [-955.0, -949.0], [-967.0, -933.0], [-968.0, -924.0], [-977.0, -919.0], [-978.0, -914.0], [-980.0, -913.0], [-983.0, -910.0], [-1001.0, -884.0], [-1002.0, -883.0], [-1004.0, -882.0], [-1021.0, -856.0], [-770.0, -1017.0], [-774.0, -1016.0], [-780.0, -1015.0], [-796.0, -1014.0], [-834.0, -1010.0], [-839.0, -1007.0], [-869.0, -998.0], [-884.0, -990.0], [-889.0, -988.0], [-890.0, -986.0], [-900.0, -982.0], [-901.0, -981.0], [-907.0, -979.0], [-932.0, -960.0], [-936.0, -958.0], [-988.0, -908.0], [-1005.0, -880.0], [-101

In [82]:
print(time1,time2)

18.875456199999917 19.372436200000266
